# Declare dependancies of the project

In [2]:
:dep itertools
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
:dep splines
:dep csv
use itertools::Itertools;
use splines::Key;
use std::{
    collections::HashMap,
    fs::File,
    hash::BuildHasher,
    io::{self, BufReader},
};
extern crate plotters;
use plotters::prelude::*;

# Data importing
All data comes from the csv files in ./data and goes from 1990 to 2021

In [3]:
const START_YEAR: usize = 1990;
const END_YEAR: usize = 2021;

Samples a data point using the given year.

In [4]:
fn value_from_year(year: usize, data: &[f64]) -> Option<f64> {
    year.checked_sub(START_YEAR)
        .and_then(|index| data.get(index).copied())
}

Load HDI data from ./data/HDI.csv [Source](https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv)

In [5]:
fn load_hdi() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/HDI.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(1).and_then(|name| {
                (5..37)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

Load GDP per capita, ppp adjusted, data from ./data/GDP.csv [Source](https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.CD?downloadformat=csv)

In [6]:
fn load_gdp() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/GDP.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(0).and_then(|name| {
                (1..33)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

## Filling in the gaps
The gini coeficint data is sparse as it is a relativly difficult indicator to measure and many countries have only a few or no data points what so ever. It seems tempting to omit countries with little data, but since this investigation is looking at development it would spoil the results to do so, as the countries with little data tend to be the ones low in thier development. The Gini coeficeint dosn't change much year on year so I think it is reasonable to interploate the availible data to do our investigation. To do this we will construct splines and sample them.

In [7]:
fn sample_spline(all: Vec<(usize, Option<f64>)>, spline: splines::Spline<f64, f64>) -> Vec<f64> {
    all.iter()
        .map(|(year, value)| {
            value.map_or_else(
                || spline.clamped_sample((*year + START_YEAR) as f64).unwrap(),
                |value| value,
            )
        })
        .collect_vec()
}

This function loads the data, and fills in the gaps with splines

In [8]:
fn load_gini() -> Option<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/Gini.csv").ok()?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Some(
        csv.records()
            .flatten()
            .filter_map(|row| {
                row.get(0).and_then(|name| {
                    let all = (1..33)
                        .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                        .enumerate()
                        .collect_vec();

                    let keys = all
                        .iter()
                        .filter_map(|(y, v)| v.map(|value| (value, (y + START_YEAR) as f64)))
                        .map(|(v, y)| Key::new(y, v, splines::Interpolation::Cosine))
                        .collect_vec();

                    if keys.is_empty() {
                        None
                    } else {
                        let spline = splines::Spline::from_vec(keys);

                        Some((name.to_string(), sample_spline(all, spline)))
                    }
                })
            })
            .collect::<HashMap<String, Vec<f64>>>(),
    )
}

## Example interpolation
Here we have the Gini data for Bangladesh. 
- The blue points are the acutal data
- The green are the interpolated points

In [9]:
let mut data: Vec<(f64,f64)> = vec![];
{
    let gini_data = load_gini().unwrap();
    let bangladesh = gini_data.get("Bangladesh").expect("Bangladesh not in data set");
    for year in START_YEAR..END_YEAR {
        data.push(((year ) as f64, value_from_year(year, bangladesh).unwrap()));
    }
}
let mut raw_data: Vec<(f64,f64)> = vec![];
{
    let file = File::open("./data/Gini.csv").ok().unwrap();
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    for record in csv.records().flatten() {
        if record.get(0).is_some_and(|name| name != "Bangladesh") {
            continue;
        }         
        raw_data = (1..33).map(|i| record.get(i).and_then(|s| s.parse::<f64>().ok())).enumerate().filter_map(|(i,v)|{
            let year = (i + START_YEAR) as f64;
            v.map(|v| (year,v))
        }).collect_vec();
    }
};



evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Gini for bangladesh", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged((START_YEAR as f64)..(END_YEAR as f64), 20.0..40.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    chart.draw_series(data.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())));
    chart.draw_series(raw_data.iter().map(|(x,y)| Circle::new((*x,*y), 3, BLUE.filled())));

    
    Ok(())
})

Gini for bangladesh
 
 
 
20.0
 
 
 
22.0
 
 
 
24.0
 
 
 
26.0
 
 
 
28.0
 
 
 
30.0
 
 
 
32.0
 
 
 
34.0
 
 
 
36.0
 
 
 
38.0
 
 
 
40.0
 
 
 
 
1990.0
 
 
 
1995.0
 
 
 
2000.0
 
 
 
2005.0
 
 
 
2010.0
 
 
 
2015.0
 
 
 
2020.0

# Using the data
First well load  the data using the functions defined before

In [10]:
let hdi_data = load_hdi().unwrap();
let gdp_data = load_gdp().unwrap();
let gini_data = load_gini().unwrap();

## Plotting
First lets plot the HDI against the Gini

In [11]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("HDI vs Gnii", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..100f64, 0.2..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    

    let mut data: Vec<(f64,f64,f64)> = vec![];
    let mut i = 0.0;
    for (name, hdi_points) in hdi_data.iter().skip(50) {
        if let Some(gini_points) = gini_data.get(name) {
            for (x,y) in gini_points.iter().zip(hdi_points.iter()) {
                data.push((*x,*y,i));
            }
        }
        i = i + 0.1 ;
    }

    let _ = chart.draw_series(data.iter().map(|(x,y,c)| Circle::new((*x,*y), 3, HSLColor(240.0 / 360.0 - 240.0 / 360.0 * c / 5.0,1.0,0.7).filled())));

    
    Ok(())
})

HDI vs Gnii
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

Not exactly crystal clear, lets plot it in 3d, letting us see how it evolves...

In [12]:
evcxr_figure((1000, 780), |root| {
    let root = root.titled("HDI vs Gini coeficent over time.", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(20.0..70.0, 0.3..0.9, 1990.0..2021.0)?;
    
        chart.with_projection(|mut p| {
            p.pitch = 0.6; 
            p.yaw = 0.4;
            p.scale = 0.7;
            p.into_matrix() // build the projection matrix
        });
    
    chart.configure_axes().draw()?;

    let mut c = 0.0;
    for (name, hdi_points) in hdi_data.iter() {
        if let Some(gini_points) = gini_data.get(name) {
            let mut line = vec![];
            for (i,(x,y)) in gini_points.iter().zip(hdi_points.iter()).enumerate() {
                line.push((*x,*y,i as f64 + START_YEAR as f64));
            }
                chart.draw_series(LineSeries::new(
        line.iter().map(|point| *point),
        &HSLColor(240.0 / 360.0 - 240.0 / 360.0 * c / 5.0,1.0,0.7),
    ))?;
        c = ( c + 0.1 ) % 5.0 ;
        }
    }  
    
    Ok(())
})

HDI vs Gini coeficent over time.
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
1990.0
 
 
 
1995.0
 
 
 
2000.0
 
 
 
2005.0
 
 
 
2010.0
 
 
 
2015.0
 
 
 
2020.0

The from the data we can see that there is no clear relationship between HDI and Gini numbers, and is therfore likely to be little corelation between development and equaltiy.
# Part 2
So the kuznets was wrong right? Well the theory is about equality, but not nessisarily wealth or income equality.
To better understand what i'm talking about, lets look at HDI vs GDP data.

In [13]:
let mut data: Vec<(f64,f64)> = vec![];
for (name, hdi_points) in hdi_data.iter() {
    if let Some(gdp_points) = gdp_data.get(name) {
        for (x,y) in gdp_points.iter().zip(hdi_points.iter()) {
            data.push((*x,*y));
        }
    }
}

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("HDI vs Real GDP PPP PC", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..10e4f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    // Draw data points
    let _ = chart.draw_series(data.iter().map(|(x,y)| Circle::new((*x,*y), 3, RGBAColor(255,100,120,0.2).filled())));
    // Draw 1 / x
    let _ = chart.draw_series(LineSeries::new(
        (0..700).map(|x| {
            let x = x as f64 ;
            let y = 1. / -(x*0.05+ 1.2) + 1.0;
            (x * 200.0 ,y * 1.)
            } ),  BLUE
    ));
    
    Ok(())
})

HDI vs Real GDP PPP PC
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
10000.0
 
 
 
20000.0
 
 
 
30000.0
 
 
 
40000.0
 
 
 
50000.0
 
 
 
60000.0
 
 
 
70000.0
 
 
 
80000.0
 
 
 
90000.0
 
 
 
100000.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


Here we can clearly see the inverse falloff relationship between development and GDP. This is caused by the same mechanism that causes marginal utiliy to fall. As countries become more developed, the additional reasouces reqired to develop further increase exponentialy. And while this is a macro example, the same principle applies at the micro level within individual countries. So despite the fact absolute wealth dispartiy may increase with develpment, it's posible that the overal equality may trend in similar fasion to the kuznetz cuvre. 

# Moddling equality

## Gini and the lorenz curve
Earlyer in the year I devised a simple expresion for plotting lorenz curves $ x^{k} $
- Where k is a constant and $ k = \frac{-2}{ G -1 } -1 $
- G is the  Gini coeficeint
This curve fufils the property $ G = \frac{A}{A+B} $ or more simply $ G = 1 - 2B $  
Example: USA with G = 0.48

In [37]:
let g = 0.48;
let k = -2.0 / (g - 1.0) - 1.0 ;

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Lorenz curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..1.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..=1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = f64::powf(x,k);
            (x  ,y )
            } ),  BLUE
    ));
    let _ = chart.draw_series(LineSeries::new(
        (0..2).map(|x| {
            let x = x as f64;
            let y = x;
            (x  ,y )
            } ),  RED
    ));
    
    Ok(())
})

Lorenz curve for USA
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,439 40,439 41,439 41,439 42,439 42,439 43,439 44,439 44,439 45,439 45,439 46,439 47,439 47,439 48,439 48,439 49,439 50,439 50,439 51,439 51,439 52,439 53,439 53,439 54,439 54,439 55,439 56,439 56,439 57,439 57,439 58,439 59,439 59,439 60,439 60,439 61,439 62,439 62,439 63,439 63,439 64,439 65,439 65,439 66,439 66,439 67,439 68,439 68,439 69,439 69,439 70,439 71,439 71,439 72,439 72,439 73,439 74,439 74,439 75,439 75,439 76,439 77,439 77,439 78,439 78,439 79,439 80,439 80,439 81,439 81,439 82,439 83,439 83,439 84,439 84,439 85,439 86,439 86,439 87,439 87,439 88,439 89,439 89,439 90,439 90,439 91,439 92,439 92,439 93,439 93,439 94,439 95,439 95,439 96,439 96,439 97,439 98,439 98,439 99,439 99,439 100,439 101,439 101,439 102,439 102,439 103,439 104,439 104,439 105,439 105,439 106,439 107,439 107,439 108,439 108,439 109,439 110,439 110,439 111,439 111,439 112,438 113,438 113,438 114,438 114,438 115,438 116,438 116,438 117,438 117,438 118,438 119,438 119,438 120,438 120,438 121,438 122,438 122,438 123,438 123,438 124,438 125,438 125,438 126,438 126,438 127,438 128,438 128,438 129,438 129,438 130,438 131,438 131,438 132,437 132,437 133,437 134,437 134,437 135,437 135,437 136,437 137,437 137,437 138,437 138,437 139,437 140,437 140,437 141,437 141,437 142,437 143,437 143,437 144,437 144,437 145,437 146,437 146,436 147,436 147,436 148,436 149,436 149,436 150,436 150,436 151,436 152,436 152,436 153,436 153,436 154,436 155,436 155,436 156,436 156,436 157,436 158,435 158,435 159,435 159,435 160,435 160,435 161,435 162,435 162,435 163,435 163,435 164,435 165,435 165,435 166,435 166,435 167,434 168,434 168,434 169,434 169,434 170,434 171,434 171,434 172,434 172,434 173,434 174,434 174,434 175,434 175,433 176,433 177,433 177,433 178,433 178,433 179,433 180,433 180,433 181,433 181,433 182,433 183,432 183,432 184,432 184,432 185,432 186,432 186,432 187,432 187,432 188,432 189,432 189,432 190,431 190,431 191,431 192,431 192,431 193,431 193,431 194,431 195,431 195,431 196,430 196,430 197,430 198,430 198,430 199,430 199,430 200,430 201,430 201,430 202,429 202,429 203,429 204,429 204,429 205,429 205,429 206,429 207,429 207,428 208,428 208,428 209,428 210,428 210,428 211,428 211,428 212,428 213,427 213,427 214,427 214,427 215,427 216,427 216,427 217,427 217,426 218,426 219,426 219,426 220,426 220,426 221,426 222,426 222,425 223,425 223,425 224,425 225,425 225,425 226,425 226,424 227,424 228,424 228,424 229,424 229,424 230,424 231,424 231,423 232,423 232,423 233,423 234,423 234,423 235,422 235,422 236,422 237,422 237,422 238,422 238,422 239,421 240,421 240,421 241,421 241,421 242,421 243,420 243,420 244,420 244,420 245,420 246,420 246,420 247,419 247,419 248,419 249,419 249,419 250,419 250,418 251,418 252,418 252,418 253,418 253,417 254,417 255,417 255,417 256,417 256,417 257,416 258,416 258,416 259,416 259,416 260,415 261,415 261,415 262,415 262,415 263,415 264,414 264,414 265,414 265,414 266,414 267,413 267,413 268,413 268,413 269,413 270,412 270,412 271,412 271,412 272,412 273,411 273,411 274,411 274,411 275,411 276,410 276,410 277,410 277,410 278,409 279,409 279,409 280,409 280,409 281,408 281,408 282,408 283,408 283,408 284,407 284,407 285,407 286,407 286,406 287,406 287,406 288,406 289,405 289,405 290,405 290,405 291,405 292,404 292,404 293,404 293,404 294,403 295,403 295,403 296,403 296,402 297,402 298,402 298,402 299,401 299,401 300,401 301,401 301,400 302,400 302,400 303,400 304,399 304,399 305,399 305,399 306,398 307,398 307,398 308,397 308,397 309,397 310,397 310,396 311,396 311,396 312,396 313,395 313,395 314,395 314,394 315,394 316,394 316,394 317,393 317,393 318,393 319,392 319,392 320,392 320,392 321,391 322,391 32

## Creating an income distribution from the lorenz curve
The lorenz curve shows cumlative values, or in other words, the intergral of the individual incomes. Therfore to reconstruct a modle of the incomes all we need to do is take the deriviative of the curve and multiply by the total amount of real wealth to get at the values. Ill be using real GDP PPP adjusted per capita to provide a stand in for this figure. The math is as follows
$ I(x) = \frac{d}{dx} x^{k} = kx^{k-1} $
Conviently the area under this curve from 0 to 1 is always 1. So to get real wealth... $ R(x) = gkx^{k-1} $ 
- Where g is GDP PPP PC
Example plot of the USA:

In [81]:
let gdp: f64 = gdp_data.get("United States").unwrap()[31];

evcxr_figure((700, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Real wealth curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..20e4)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = gdp*k*x.powf( k - 1.0 );
            (x  ,y )
            } ),  GREEN
    ));
    
    Ok(())
})

Real wealth curve for USA
 
 
 
0.0
 
 
 
20000.0
 
 
 
40000.0
 
 
 
60000.0
 
 
 
80000.0
 
 
 
100000.0
 
 
 
120000.0
 
 
 
140000.0
 
 
 
160000.0
 
 
 
180000.0
 
 
 
200000.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="40,259 40,259 41,259 41,259 42,259 43,259 43,259 44,259 45,259 45,259 46,259 47,259 47,259 48,259 49,259 49,259 50,259 51,259 51,259 52,259 53,259 53,259 54,259 55,259 55,259 56,259 57,259 57,259 58,259 59,259 59,259 60,259 61,259 61,259 62,259 63,259 63,259 64,259 65,259 65,259 66,259 67,259 67,259 68,259 68,259 69,259 70,259 70,259 71,259 72,259 72,259 73,259 74,259 74,258 75,258 76,258 76,258 77,258 78,258 78,258 79,258 80,258 80,258 81,258 82,258 82,258 83,258 84,258 84,258 85,258 86,258 86,258 87,258 88,258 88,258 89,258 90,257 90,257 91,257 92,257 92,257 93,257 94,257 94,257 95,257 96,257 96,257 97,257 97,257 98,257 99,257 99,257 100,257 101,257 101,257 102,256 103,256 103,256 104,256 105,256 105,256 106,256 107,256 107,256 108,256 109,256 109,256 110,256 111,256 111,256 112,256 113,255 113,255 114,255 115,255 115,255 116,255 117,255 117,255 118,255 119,255 119,255 120,255 121,255 121,255 122,254 123,254 123,254 124,254 125,254 125,254 126,254 126,254 127,254 128,254 128,254 129,254 130,254 130,253 131,253 132,253 132,253 133,253 134,253 134,253 135,253 136,253 136,253 137,253 138,253 138,252 139,252 140,252 140,252 141,252 142,252 142,252 143,252 144,252 144,252 145,252 146,252 146,251 147,251 148,251 148,251 149,251 150,251 150,251 151,251 152,251 152,251 153,250 154,250 154,250 155,250 155,250 156,250 157,250 157,250 158,250 159,250 159,249 160,249 161,249 161,249 162,249 163,249 163,249 164,249 165,249 165,249 166,248 167,248 167,248 168,248 169,248 169,248 170,248 171,248 171,248 172,247 173,247 173,247 174,247 175,247 175,247 176,247 177,247 177,247 178,246 179,246 179,246 180,246 181,246 181,246 182,246 183,246 183,246 184,245 184,245 185,245 186,245 186,245 187,245 188,245 188,245 189,244 190,244 190,244 191,244 192,244 192,244 193,244 194,244 194,243 195,243 196,243 196,243 197,243 198,243 198,243 199,243 200,242 200,242 201,242 202,242 202,242 203,242 204,242 204,241 205,241 206,241 206,241 207,241 208,241 208,241 209,241 210,240 210,240 211,240 212,240 212,240 213,240 213,240 214,239 215,239 215,239 216,239 217,239 217,239 218,239 219,238 219,238 220,238 221,238 221,238 222,238 223,238 223,237 224,237 225,237 225,237 226,237 227,237 227,237 228,236 229,236 229,236 230,236 231,236 231,236 232,236 233,235 233,235 234,235 235,235 235,235 236,235 237,234 237,234 238,234 239,234 239,234 240,234 240,233 241,233 242,233 242,233 243,233 244,233 244,233 245,232 246,232 246,232 247,232 248,232 248,232 249,231 250,231 250,231 251,231 252,231 252,231 253,230 254,230 254,230 255,230 256,230 256,230 257,229 258,229 258,229 259,229 260,229 260,229 261,228 262,228 262,228 263,228 264,228 264,228 265,227 266,227 266,227 267,227 268,227 268,227 269,226 269,226 270,226 271,226 271,226 272,225 273,225 273,225 274,225 275,225 275,225 276,224 277,224 277,224 278,224 279,224 279,223 280,223 281,223 281,223 282,223 283,223 283,222 284,222 285,222 285,222 286,222 287,221 287,221 288,221 289,221 289,221 290,220 291,220 291,220 292,220 293,220 293,220 294,219 295,219 295,219 296,219 297,219 297,218 298,218 298,218 299,218 300,218 300,217 301,217 302,217 302,217 303,217 304,216 304,216 305,216 306,216 306,216 307,215 308,215 308,215 309,215 310,215 310,214 311,214 312,214 312,214 313,214 314,213 314,213 315,213 316,213 316,213 317,212 318,212 318,212 319,212 320,212 320,211 321,211 322,211 322,211 323,210 324,210 324,210 325,210 326,210 326,209 327,209 327,209 328,209 329,209 329,208 330,208 331,208 331,208 332,207 333,207 333,207 334,207 335,207 335,206 336,206 337,206 337,206 338,206 339,205 339,205 340,205 341,205 341,204 342,204 343,204 343,204 344,204 345,203 345,20

## Moddling utility

From the law of decreasing marginal utility we know
$ \Delta U = e^{kc} $ where 
- U is utility
- c is conumption or any other source of utility
- k is some negative constant

And therfore U as the intergral of marginal uitilty is $ U = \int_{0}^{c} e^{kc} \, dc =  \frac{e^{kc}-1}{k} $


## Putting it together
Now we have the quantity of real wealth, and model of how that translates to utility. We'll now combine them by inserting the former into the latter  
### $\frac{ e^{ Ugkx^{ k - 1 } } - 1 }{ U }$
Example plot where $ U = -0.0001 $

In [80]:
let u = -0.0001;

evcxr_figure((700, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Utility wealth curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..10000.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = ( ( gdp * u * k * x.powf(k - 1.0) ).exp() - 1.0 ) / u ;
            (x  ,y )
            } ),  GREEN
    ));
    
    Ok(())
})

Utility wealth curve for USA
 
 
 
0.0
 
 
 
1000.0
 
 
 
2000.0
 
 
 
3000.0
 
 
 
4000.0
 
 
 
5000.0
 
 
 
6000.0
 
 
 
7000.0
 
 
 
8000.0
 
 
 
9000.0
 
 
 
10000.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="40,259 40,259 41,259 41,259 42,259 43,259 43,259 44,259 45,259 45,259 46,259 47,258 47,258 48,258 49,258 49,258 50,257 51,257 51,257 52,256 53,256 53,256 54,255 55,255 55,255 56,254 57,254 57,254 58,253 59,253 59,252 60,252 61,252 61,251 62,251 63,250 63,250 64,249 65,249 65,248 66,248 67,247 67,246 68,246 68,245 69,245 70,244 70,244 71,243 72,242 72,242 73,241 74,240 74,240 75,239 76,238 76,238 77,237 78,236 78,236 79,235 80,234 80,234 81,233 82,232 82,231 83,231 84,230 84,229 85,228 86,228 86,227 87,226 88,225 88,224 89,224 90,223 90,222 91,221 92,220 92,219 93,219 94,218 94,217 95,216 96,215 96,214 97,213 97,213 98,212 99,211 99,210 100,209 101,208 101,207 102,206 103,205 103,205 104,204 105,203 105,202 106,201 107,200 107,199 108,198 109,197 109,196 110,195 111,194 111,193 112,192 113,191 113,191 114,190 115,189 115,188 116,187 117,186 117,185 118,184 119,183 119,182 120,181 121,180 121,179 122,178 123,177 123,176 124,175 125,174 125,173 126,172 126,171 127,170 128,169 128,168 129,167 130,167 130,166 131,165 132,164 132,163 133,162 134,161 134,160 135,159 136,158 136,157 137,156 138,155 138,154 139,153 140,152 140,151 141,150 142,149 142,148 143,148 144,147 144,146 145,145 146,144 146,143 147,142 148,141 148,140 149,139 150,138 150,137 151,137 152,136 152,135 153,134 154,133 154,132 155,131 155,130 156,129 157,129 157,128 158,127 159,126 159,125 160,124 161,123 161,122 162,122 163,121 163,120 164,119 165,118 165,117 166,117 167,116 167,115 168,114 169,113 169,113 170,112 171,111 171,110 172,109 173,109 173,108 174,107 175,106 175,106 176,105 177,104 177,103 178,102 179,102 179,101 180,100 181,100 181,99 182,98 183,97 183,97 184,96 184,95 185,95 186,94 186,93 187,92 188,92 188,91 189,90 190,90 190,89 191,88 192,88 192,87 193,86 194,86 194,85 195,85 196,84 196,83 197,83 198,82 198,81 199,81 200,80 200,80 201,79 202,78 202,78 203,77 204,77 204,76 205,76 206,75 206,74 207,74 208,73 208,73 209,72 210,72 210,71 211,71 212,70 212,70 213,69 213,69 214,68 215,68 215,67 216,67 217,66 217,66 218,65 219,65 219,64 220,64 221,63 221,63 222,62 223,62 223,62 224,61 225,61 225,60 226,60 227,59 227,59 228,59 229,58 229,58 230,57 231,57 231,57 232,56 233,56 233,55 234,55 235,55 235,54 236,54 237,54 237,53 238,53 239,53 239,52 240,52 240,52 241,51 242,51 242,51 243,50 244,50 244,50 245,49 246,49 246,49 247,48 248,48 248,48 249,47 250,47 250,47 251,47 252,46 252,46 253,46 254,45 254,45 255,45 256,45 256,44 257,44 258,44 258,44 259,43 260,43 260,43 261,43 262,42 262,42 263,42 264,42 264,42 265,41 266,41 266,41 267,41 268,40 268,40 269,40 269,40 270,40 271,39 271,39 272,39 273,39 273,39 274,38 275,38 275,38 276,38 277,38 277,38 278,37 279,37 279,37 280,37 281,37 281,37 282,36 283,36 283,36 284,36 285,36 285,36 286,36 287,35 287,35 288,35 289,35 289,35 290,35 291,35 291,34 292,34 293,34 293,34 294,34 295,34 295,34 296,34 297,33 297,33 298,33 298,33 299,33 300,33 300,33 301,33 302,33 302,32 303,32 304,32 304,32 305,32 306,32 306,32 307,32 308,32 308,32 309,32 310,31 310,31 311,31 312,31 312,31 313,31 314,31 314,31 315,31 316,31 316,31 317,31 318,31 318,30 319,30 320,30 320,30 321,30 322,30 322,30 323,30 324,30 324,30 325,30 326,30 326,30 327,30 327,30 328,29 329,29 329,29 330,29 331,29 331,29 332,29 333,29 333,29 334,29 335,29 335,29 336,29 337,29 337,29 338,29 339,29 339,29 340,29 341,29 341,29 342,29 343,28 343,28 344,28 345,28 345,28 346,28 347,28 347,28 348,28 349,28 349,28 350,28 351,28 351,28 352,28 353,28 353,28 354,28 355,28 355,28 356,28 356,28 357,28 358,28 358,28 359,28 360,28 360,28 361,28 362,28 362,28 363,28 364,28 364,28 365,28 366,27 366,27 367,27 368,27 368,27 369,27 370,27 3

As you can see, the power function of income combined with the fall of utility produce a sigmoid like curve.
## Measuing the inequality of the curve
In order to measure the equality of this curve we just need to reintergrate it back into a lorenz curve...  
# $ \frac
{ \int_{0}^{x} e^{Ugkx^{k-1}} \,dx}
{ \int_{0}^{1} e^{Ugkx^{k-1}} \,dx}$  
I was suprised to find that intergrals like these don't have anilytic solutions. Luckly we live in the 21st century and have no need to do such archic things like doing maths by hand. The computer will easly be able to solve them numericaly.

In [90]:
let u = -0.0001;

evcxr_figure((700, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Utility lorenz curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..1.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    let mut total = 0.0;
    let slices = 1000.0;
    let dx = 1.0 / slices; 
    
    let mut data = Vec::with_capacity(1000);
    let mut x: f64 = 0.0;
    while x < 1.0 {
        total += ( ( gdp * u * k * x.powf(k - 1.0) ).exp() - 1.0 ) / u  * dx;
        data.push((x,total));
        x += dx;
    }
    
    println!("Total: {total}");
    let _ = chart.draw_series(LineSeries::new(
        data.iter().map(|(x,y)| {
            let y = y / total;
            (*x  ,y )
            } ),  GREEN
    ));
    
    
     let _ = chart.draw_series(LineSeries::new(
        (0..2).map(|x| {
            let x = x as f64;
            let y = x;
            (x  ,y )
            } ),  RED
    ));
    
    Ok(())
})

Total: 8241.123804964629


Utility lorenz curve for USA
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="40,259 40,259 41,259 41,259 42,259 43,259 43,259 44,259 45,259 45,259 46,259 47,259 47,259 48,259 49,259 49,259 50,259 51,259 51,259 52,259 53,259 53,259 54,259 55,259 55,259 56,259 57,259 57,259 58,259 59,259 59,259 60,259 61,259 61,259 62,259 63,259 63,259 64,259 65,259 65,259 66,259 67,259 67,259 68,259 68,259 69,259 70,259 70,259 71,259 72,259 72,259 73,259 74,259 74,259 75,259 76,259 76,259 77,259 78,259 78,259 79,259 80,259 80,259 81,259 82,259 82,259 83,259 84,259 84,259 85,259 86,258 86,258 87,258 88,258 88,258 89,258 90,258 90,258 91,258 92,258 92,258 93,258 94,258 94,258 95,258 96,258 96,258 97,258 97,258 98,258 99,258 99,257 100,257 101,257 101,257 102,257 103,257 103,257 104,257 105,257 105,257 106,257 107,257 107,257 108,257 109,256 109,256 110,256 111,256 111,256 112,256 113,256 113,256 114,256 115,256 115,256 116,256 117,255 117,255 118,255 119,255 119,255 120,255 121,255 121,255 122,255 123,255 123,255 124,254 125,254 125,254 126,254 126,254 127,254 128,254 128,254 129,254 130,253 130,253 131,253 132,253 132,253 133,253 134,253 134,253 135,253 136,252 136,252 137,252 138,252 138,252 139,252 140,252 140,252 141,251 142,251 142,251 143,251 144,251 144,251 145,251 146,250 146,250 147,250 148,250 148,250 149,250 150,250 150,249 151,249 152,249 152,249 153,249 154,249 154,248 155,248 155,248 156,248 157,248 157,248 158,248 159,247 159,247 160,247 161,247 161,247 162,247 163,246 163,246 164,246 165,246 165,246 166,246 167,245 167,245 168,245 169,245 169,245 170,244 171,244 171,244 172,244 173,244 173,244 174,243 175,243 175,243 176,243 177,243 177,242 178,242 179,242 179,242 180,242 181,241 181,241 182,241 183,241 183,241 184,240 184,240 185,240 186,240 186,240 187,239 188,239 188,239 189,239 190,239 190,238 191,238 192,238 192,238 193,238 194,237 194,237 195,237 196,237 196,237 197,236 198,236 198,236 199,236 200,235 200,235 201,235 202,235 202,235 203,234 204,234 204,234 205,234 206,233 206,233 207,233 208,233 208,233 209,232 210,232 210,232 211,232 212,231 212,231 213,231 213,231 214,231 215,230 215,230 216,230 217,230 217,229 218,229 219,229 219,229 220,228 221,228 221,228 222,228 223,227 223,227 224,227 225,227 225,226 226,226 227,226 227,226 228,226 229,225 229,225 230,225 231,225 231,224 232,224 233,224 233,224 234,223 235,223 235,223 236,223 237,222 237,222 238,222 239,222 239,221 240,221 240,221 241,221 242,220 242,220 243,220 244,220 244,219 245,219 246,219 246,218 247,218 248,218 248,218 249,217 250,217 250,217 251,217 252,216 252,216 253,216 254,216 254,215 255,215 256,215 256,215 257,214 258,214 258,214 259,214 260,213 260,213 261,213 262,213 262,212 263,212 264,212 264,211 265,211 266,211 266,211 267,210 268,210 268,210 269,210 269,209 270,209 271,209 271,209 272,208 273,208 273,208 274,207 275,207 275,207 276,207 277,206 277,206 278,206 279,206 279,205 280,205 281,205 281,204 282,204 283,204 283,204 284,203 285,203 285,203 286,203 287,202 287,202 288,202 289,202 289,201 290,201 291,201 291,200 292,200 293,200 293,200 294,199 295,199 295,199 296,199 297,198 297,198 298,198 298,197 299,197 300,197 300,197 301,196 302,196 302,196 303,195 304,195 304,195 305,195 306,194 306,194 307,194 308,194 308,193 309,193 310,193 310,192 311,192 312,192 312,192 313,191 314,191 314,191 315,190 316,190 316,190 317,190 318,189 318,189 319,189 320,189 320,188 321,188 322,188 322,187 323,187 324,187 324,187 325,186 326,186 326,186 327,185 327,185 328,185 329,185 329,184 330,184 331,184 331,184 332,183 333,183 333,183 334,182 335,182 335,182 336,182 337,181 337,181 338,181 339,180 339,180 340,180 341,180 341,179 342,179 343,179 343,178 344,178 345,178 345,178 346,177 347,177 347,177 348,177 349,176 3